In [26]:
!pip install pytelegrambotapi

In [27]:
import telebot
import os

In [28]:
os.environ['FOLDER_ID'] = "%FOLDER_ID%"
os.environ['IAM_TOKEN'] = "%IAM_TOKEN%"

In [29]:
%%writefile KeyIdea_ogg.py

import argparse
import requests


def synthesize(folder_id, iam_token, text):
    url = 'https://tts.api.cloud.yandex.net/speech/v1/tts:synthesize'
    headers = {
        'Authorization': 'Bearer ' + iam_token,
    }

    data = {
        'text': text,
        'lang': 'ru-RU',
        'folderId': folder_id
    }

    with requests.post(url, headers=headers, data=data, stream=True) as resp:
        if resp.status_code != 200:
            raise RuntimeError("Invalid response received: code: %d, message: %s" % (resp.status_code, resp.text))

        for chunk in resp.iter_content(chunk_size=None):
            yield chunk


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--token", required=True, help="IAM token")
    parser.add_argument("--folder_id", required=True, help="Folder id")
    parser.add_argument("--text", required=True, help="Text for synthesize")
    parser.add_argument("--output", required=True, help="Output file name")
    args = parser.parse_args()

    with open(args.output, "wb") as f:
        for audio_content in synthesize(args.folder_id, args.token, args.text):
            f.write(audio_content)

Writing KeyIdea_ogg.py


In [30]:
def recognize(file_):
  import urllib.request
  import json

  FOLDER_ID = os.environ['FOLDER_ID']
  IAM_TOKEN = os.environ['IAM_TOKEN']

  with open(file_, "rb") as f:
      data = f.read()

  params = "&".join([
    "topic=general",
    "folderId=%s" % FOLDER_ID,
    "lang=ru-RU"
  ])

  url = urllib.request.Request("https://stt.api.cloud.yandex.net/speech/v1/stt:recognize?%s" % params, data=data)
  url.add_header("Authorization", "Bearer %s" % IAM_TOKEN)

  responseData = urllib.request.urlopen(url).read().decode('UTF-8')
  decodedData = json.loads(responseData)

  if decodedData.get("error_code") is None:
    return decodedData.get("result")

In [31]:
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech_default.ogg --text "Максим, я не вполне вас поняла... Давайте заново!"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech1.ogg --text "Максим, привет! У тебя 200 писем не прочитано. Начнем их разбирать?"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech2.ogg --text "Письмо первое от Дмитрия Иванова. Рассказать, кто это?"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech3.ogg --text "Ведет разработку Digital Manager, важность писем от него не самая высокая. В меру лаконичен, пишет грамотно. Связаться с ним?"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech4.ogg --text "Хорошо. Прочитать краткое содержание письма?"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech5.ogg --text "Тема: Digital Manager, новый релиз. Содержание: Мы выпустили новый релиз Digital Manager. Ключевые улучшения: еще больше багов, еще более странные ответы от gpt-3, отправлено 19.11.2020. Рассказать о том, какие инсайты мы нашли о письме?"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech6.ogg --text "Письмо писало 3 автора, вероятно Дмитрий Иванов, Михаил Муравьев и Сергей Волков, лаконичноть невысокая, но высокая грамотность. Это первое письмо в цепочке, так что скорость ответа не рассчитана. Зачитать полную версию письма?"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech7.ogg --text "Если хотите ответить на письмо - продиктуйте ответ, либо скажите 'не отвечать', чтобы оставить без ответа. Также можно сказать 'ЭТО ТРЭШ', чтобы автоматически закончить работу с письмом"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech8.ogg --text "Спасибо, ответ отправлен. Хотите делегировать это письмо Андрею Свиридову? Либо скажите 'не делегировать', или скажите 'делегировать тому-то'"
!python KeyIdea_ogg.py --token $IAM_TOKEN --folder_id $FOLDER_ID --output speech9.ogg --text "Спасибо, письмо переслано. Сценарий окончен"

In [32]:
API_TOKEN = '%API_TOKEN%'
bot = telebot.TeleBot(API_TOKEN)

counter=1

@bot.message_handler(content_types=['voice'], func=lambda message: message.from_user.id == 230968360)
def echo_messages(message):

    next_fl = False
    global counter
    print(message.from_user)


    if counter == 1:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      if any(x in result.lower() for x in ["привет", "здравствуйте", "здравствуй"]):
        #Максим, привет! У тебя 200 писем не прочитано. Начнем их разбирать?
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        

    if counter == 2:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["да", "давай", "окей", "поехали", "погнали"]):
        #Письмо первое от Дмитрия Иванова. Рассказать, кто это?
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)


    if counter == 3:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["да", "давай", "окей", "поехали", "погнали", "расскажи", "рассказывай"]):
        #Ведет разработку Digital Manager, важность писем от него не самая высокая. В меру лаконичен, пишет грамотно. Связаться с ним?
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)

    if counter == 4:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["нет", "не надо", "дальше"]):
        #Хорошо. Прочитать краткое содержание письма?
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)

    if counter == 5:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["да", "давай", "окей", "поехали", "погнали", "читай", "прочитай"]):
        #Тема: Digital Manager, новый релиз. Содержание: Мы выпустили новый релиз Digital Manager. Ключевые улучшения: еще больше багов, еще более странные ответы от gpt-3, отправлено 19.11.2020. Рассказать о том, какие инсайты мы нашли о писме?
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)

    if counter == 6:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["да", "давай", "окей", "поехали", "погнали", "расскажи", "рассказывай"]):
        #Письмо писало 3 автора, вероятно Дмитрий Иванов, Михаил Муравьев и Сергей Волков, лаконичноть невысокая, но высокая грамотность. Это первое письмо в цепочке, так что скорость ответа не рассчитана. Зачитать полную версию письма?
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)

    if counter == 7:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["нет", "не надо", "дальше"]):
        #Если хотите ответить на письмо - продиктуйте ответ, либо скажите 'не отвечать', чтобы оставить без ответа. Также можно сказать 'ЭТО ТРЭШ', чтобы автоматически закончить работу с письмом
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)

    if counter == 8:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["ок", "спасибо"]):
        #Спасибо, ответ отправлен. Хотите делегировать это письмо Андрею Свиридову? Либо скажите 'не делегировать', или скажите 'делегировать тому-то'
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = True
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)

    if counter == 9:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      result = recognize('new_file.ogg')
      
      if any(x in result.lower() for x in ["делегировать", "Чертоку"]):
        #Спасибо, письмо переслано
        voice = open('speech%d.ogg'%counter, 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)
        next_fl = False
        counter=1
      else:
        voice = open('speech_default.ogg', 'rb')
        chat_id = message.chat.id
        bot.send_voice(chat_id, voice)

      
    if next_fl:
      counter+=1

In [ ]:
bot.polling()